# **Розширений план з контентом та кодом для практичного заняття**  
## **Тема 3. Заняття 5. Практичний аналіз часових рядів**  
### **1. Аналіз часових рядів засобами машинного навчання**  
### **2. Аналіз часових рядів засобами глибокого навчання**  

---

## **1. Вступ (10 хв)**  

### **1.1. Мета заняття**  
- Ознайомити слухачів із методами аналізу часових рядів у машинному та глибокому навчанні.  
- Освоїти основні алгоритми передбачення майбутніх значень у часових рядах.  
- Виконати практичне прогнозування на реальних даних.  

### **1.2. Очікувані результати**  
Після заняття слухачі зможуть:  
- Виконувати передобробку часових рядів.  
- Використовувати машинне навчання (`ARIMA`, `Random Forest`) для аналізу часових рядів.  
- Застосовувати глибокі нейромережі (`LSTM`, `GRU`) для прогнозування часових рядів.  
- Візуалізувати результати аналізу.  

---

## **2. Аналіз часових рядів засобами машинного навчання (60 хв)**  

### **2.1. Основи часових рядів**  
1. **Що таке часовий ряд?**  
   - Часовий ряд — це послідовність значень, відсортованих за часом.  
   - Приклади: температурні дані, фінансові ринки, військові операції.  

2. **Основні характеристики:**  
   - **Стаціонарність** – розподіл даних не змінюється з часом.  
   - **Тренд** – загальний напрямок зміни (зростання/спад).  
   - **Сезонність** – періодичні коливання.  

---

### **2.2. Передобробка даних**  
```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Завантаження даних
df = pd.read_csv("time_series_data.csv", parse_dates=["date"], index_col="date")

# Перевірка пропущених значень
df.isnull().sum()

# Заповнення пропущених значень методом лінійної інтерполяції
df.fillna(method="ffill", inplace=True)

# Візуалізація часового ряду
plt.figure(figsize=(12,5))
plt.plot(df["value"], label="Часовий ряд")
plt.legend()
plt.show()
```

---

### **2.3. Аналіз часових рядів методом ARIMA**  
```python
from statsmodels.tsa.arima.model import ARIMA

# Розбиття на тренувальну та тестову вибірки
train_size = int(len(df) * 0.8)
train, test = df[:train_size], df[train_size:]

# Побудова моделі ARIMA
model = ARIMA(train["value"], order=(5,1,0))  # (p,d,q) - параметри моделі
model_fit = model.fit()

# Прогнозування
forecast = model_fit.forecast(steps=len(test))
test["forecast_ARIMA"] = forecast

# Візуалізація результатів
plt.figure(figsize=(12,5))
plt.plot(train["value"], label="Тренувальні дані")
plt.plot(test["value"], label="Тестові дані")
plt.plot(test["forecast_ARIMA"], label="Прогноз ARIMA", linestyle="dashed")
plt.legend()
plt.show()
```

---

### **2.4. Аналіз часових рядів методом Random Forest**  
```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Створення нових ознак (лагів)
df["lag1"] = df["value"].shift(1)
df["lag2"] = df["value"].shift(2)
df.dropna(inplace=True)

# Розбиття на тренувальну та тестову вибірки
X = df[["lag1", "lag2"]]
y = df["value"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Побудова моделі Random Forest
model_rf = RandomForestRegressor(n_estimators=100)
model_rf.fit(X_train, y_train)

# Прогнозування
y_pred_rf = model_rf.predict(X_test)

# Візуалізація
plt.figure(figsize=(12,5))
plt.plot(y_test.values, label="Реальні дані")
plt.plot(y_pred_rf, label="Прогноз RF", linestyle="dashed")
plt.legend()
plt.show()
```

---

## **3. Аналіз часових рядів засобами глибокого навчання (60 хв)**  

### **3.1. Використання LSTM для прогнозування**  
```python
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Нормалізація даних
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[["value"]])

# Підготовка даних для LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 10
X, y = create_sequences(df_scaled, seq_length)

# Розбиття на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Побудова LSTM моделі
model_lstm = Sequential([
    LSTM(50, activation="relu", return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, activation="relu"),
    Dense(1)
])

model_lstm.compile(optimizer="adam", loss="mse")
model_lstm.fit(X_train, y_train, epochs=20, batch_size=16)

# Прогнозування
y_pred_lstm = model_lstm.predict(X_test)
y_pred_lstm = scaler.inverse_transform(y_pred_lstm)

# Візуалізація результатів
plt.figure(figsize=(12,5))
plt.plot(y_test, label="Реальні дані")
plt.plot(y_pred_lstm, label="Прогноз LSTM", linestyle="dashed")
plt.legend()
plt.show()
```

---

### **3.2. Використання GRU для прогнозування**  
```python
from tensorflow.keras.layers import GRU

# Побудова GRU моделі
model_gru = Sequential([
    GRU(50, activation="relu", return_sequences=True, input_shape=(seq_length, 1)),
    GRU(50, activation="relu"),
    Dense(1)
])

model_gru.compile(optimizer="adam", loss="mse")
model_gru.fit(X_train, y_train, epochs=20, batch_size=16)

# Прогнозування
y_pred_gru = model_gru.predict(X_test)
y_pred_gru = scaler.inverse_transform(y_pred_gru)

# Візуалізація результатів
plt.figure(figsize=(12,5))
plt.plot(y_test, label="Реальні дані")
plt.plot(y_pred_gru, label="Прогноз GRU", linestyle="dashed")
plt.legend()
plt.show()
```

---

## **4. Практичне завдання (30 хв)**  
1. Використати **ARIMA** для прогнозування тренду часового ряду.  
2. Використати **Random Forest** для аналізу сезонності.  
3. Навчити **LSTM** на новому наборі даних.  
4. Використати **GRU** для прогнозування майбутніх значень.  

---

## **5. Висновки та підсумки (20 хв)**  
✅ **Основні висновки**:  
- **Машинне навчання** добре підходить для короткострокового прогнозування.  
- **Глибоке навчання** дає кращі результати на довгострокових часових рядах.  
- **Комбінація методів** (LSTM + Random Forest) покращує точність прогнозів.  

🚀 **Ключовий меседж**: Аналіз часових рядів є важливим для прийняття рішень у військовій аналітиці, фінансах та прогнозуванні подій.

# **Додатковий матеріал: Детальний опис моделей для аналізу часових рядів**

## **1. Класифікація моделей аналізу часових рядів**
Моделі для аналізу часових рядів можна поділити на дві великі категорії:
- **Класичні статистичні методи**: ARIMA, SARIMA, ETS.
- **Методи машинного та глибокого навчання**: Random Forest, LSTM, GRU.

Ці моделі мають різні переваги та обмеження, які визначають їх застосування у військовій аналітиці, прогнозуванні подій та економічному аналізі.

---

## **2. Класичні моделі часових рядів**

### **2.1. ARIMA (Autoregressive Integrated Moving Average)**
Модель ARIMA (Авторегресійна інтегрована модель ковзного середнього) широко використовується для прогнозування часових рядів.

🔹 **Компоненти моделі ARIMA(p,d,q)**:
- **p** – кількість авторегресійних лагів (AR).
- **d** – кількість диференціювань для отримання стаціонарного ряду.
- **q** – порядок моделі ковзного середнього (MA).

📌 **Основна ідея**:  
- Використання попередніх значень (`AR`).
- Усунення трендів за допомогою диференціювання (`I`).
- Використання минулих помилок для корекції прогнозу (`MA`).

📌 **Приклад побудови моделі ARIMA**:
```python
from statsmodels.tsa.arima.model import ARIMA

# Визначення параметрів (p,d,q)
p, d, q = 5, 1, 0

# Створення та навчання моделі
model = ARIMA(train["value"], order=(p, d, q))
model_fit = model.fit()

# Прогнозування
forecast = model_fit.forecast(steps=len(test))
test["forecast_ARIMA"] = forecast
```

🔹 **Обмеження ARIMA**:
- Підходить лише для стаціонарних рядів.
- Не враховує сезонність.

---

### **2.2. Random Forest для прогнозування часових рядів**
📌 **Ідея методу**:
- Використовує ансамбль рішень (дерева) для передбачення майбутніх значень.
- Додає випадковість до вибору ознак для навчання.

🔹 **Переваги**:
- Добре працює з нелінійними даними.
- Менш чутливий до шуму в даних.

📌 **Приклад застосування Random Forest**:
```python
from sklearn.ensemble import RandomForestRegressor

# Створення лагових ознак
df["lag1"] = df["value"].shift(1)
df["lag2"] = df["value"].shift(2)
df.dropna(inplace=True)

# Розбиття на тренувальний та тестовий набір
X = df[["lag1", "lag2"]]
y = df["value"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Побудова моделі
model_rf = RandomForestRegressor(n_estimators=100)
model_rf.fit(X_train, y_train)

# Прогнозування
y_pred_rf = model_rf.predict(X_test)
```

🔹 **Обмеження Random Forest**:
- Погано працює з довгостроковими прогнозами.
- Не враховує трендові та сезонні компоненти.

---

## **3. Глибоке навчання для часових рядів**

### **3.1. LSTM (Long Short-Term Memory)**
LSTM (Довготривала короткочасна пам’ять) – це вдосконалена версія рекурентних нейромереж, яка може зберігати інформацію про попередні події протягом довгого часу.

📌 **Основна ідея**:
- LSTM має механізм «комірок пам’яті», які керують тим, які дані потрібно зберігати та забувати.
- Використовується для довготривалого аналізу часових рядів.

🔹 **Переваги LSTM**:
- Добре підходить для довгострокового прогнозування.
- Може працювати з трендовими та сезонними даними.

📌 **Приклад моделі LSTM**:
```python
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Підготовка даних
seq_length = 10
X, y = create_sequences(df_scaled, seq_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Побудова LSTM моделі
model_lstm = Sequential([
    LSTM(50, activation="relu", return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, activation="relu"),
    Dense(1)
])

model_lstm.compile(optimizer="adam", loss="mse")
model_lstm.fit(X_train, y_train, epochs=20, batch_size=16)

# Прогнозування
y_pred_lstm = model_lstm.predict(X_test)
```

🔹 **Обмеження LSTM**:
- Висока вимогливість до обчислювальних ресурсів.
- Чутливість до гіперпараметрів.

---

### **3.2. GRU (Gated Recurrent Unit)**
GRU – це спрощена версія LSTM, яка працює швидше, але при цьому зберігає пам’ять про довгі послідовності.

📌 **Основна відмінність GRU від LSTM**:
- GRU має лише два гейти (оновлення та скидання), на відміну від трьох у LSTM.
- Швидша обробка даних, зберігаючи ефективність прогнозування.

📌 **Приклад моделі GRU**:
```python
from tensorflow.keras.layers import GRU

# Побудова GRU моделі
model_gru = Sequential([
    GRU(50, activation="relu", return_sequences=True, input_shape=(seq_length, 1)),
    GRU(50, activation="relu"),
    Dense(1)
])

model_gru.compile(optimizer="adam", loss="mse")
model_gru.fit(X_train, y_train, epochs=20, batch_size=16)

# Прогнозування
y_pred_gru = model_gru.predict(X_test)
```

🔹 **Переваги GRU**:
- Легша архітектура, ніж LSTM.
- Менше обчислювальне навантаження.
- Добре підходить для роботи з короткими часовими рядами.

🔹 **Обмеження GRU**:
- Не завжди краще за LSTM для довгих часових рядів.

---

## **4. Порівняння моделей**

| **Метод**  | **Сильні сторони** | **Обмеження** |
|------------|------------------|--------------|
| **ARIMA** | Хороший для стаціонарних рядів, точний у короткострокових прогнозах | Погано працює з нестабільними та сезонними даними |
| **Random Forest** | Добре працює з нелінійними зв’язками, легко інтерпретується | Не підходить для довгострокового прогнозування |
| **LSTM** | Ефективний для складних часових рядів, може враховувати залежності | Високі обчислювальні вимоги |
| **GRU** | Швидший за LSTM, добре працює з короткими рядами | Менш ефективний для довгих часових рядів |

---

## **5. Висновки**
✅ **ARIMA та Random Forest** підходять для класичного аналізу часових рядів.  
✅ **LSTM та GRU** показують високу ефективність у прогнозуванні складних часових рядів.  
✅ **Вибір моделі залежить від структури даних та завдання**.  

🚀 **Ключовий меседж**: Використання AI та машинного навчання в аналізі часових рядів дозволяє значно покращити прогнозування та прийняття рішень у військових аналітичних системах та економічному плануванні.